In [1]:
from pennylane import numpy as np
import pennylane as qml
from GS_training import * 

import openfermion

In [2]:
def eXXpYY(theta, wires):
    # wires = [0,1]
    X = np.array([[0, 1], [1, 0]])  
    Y = np.array([[0, -1j], [1j, 0]])
    XXpYY2=(np.kron(X, X) + np.kron(Y, Y))/2
    d, V = np.linalg.eig(XXpYY2)
    # print(np.linalg.inv(V)@XXpYY@V)
    U = V@np.diag(np.exp((0+1j)*theta*d))@np.linalg.inv(V)
    qml.QubitUnitary(U, wires=wires)
    
def eYXmXY(theta, wires):
    # wires = [0,1]
    X = np.array([[0, 1], [1, 0]])  
    Y = np.array([[0, -1j], [1j, 0]])
    YXmXY2 =-(np.kron(X, Y) - np.kron(Y, X))/2
    d, V = np.linalg.eig(YXmXY2)
    U = V@np.diag(np.exp((0+1j)*theta*d))@np.linalg.inv(V)
    qml.QubitUnitary(U, wires=wires)
def eM3(theta, wires):
    U = np.array([[0,0,0,0], [0,np.exp(-1j*theta),0,0],[0,0,-np.exp(1j*theta),0],[0,0,0,0]])
    qml.QubitUnitary(U, wires= wires)
    
def circuit(param, # p X (N+N) thetas and phis  
            w #wires list 
            ):
    
    print(param.shape)
    p = len(param)
    # (p, _)  = param.shape
    N = len(w)
    # parametrized circuit with depth p 
    for step in range(p):
        for i in range(N):
            qml.RZ(param[step,0, i], wires = w[i]) # exp(-i X phi/2)
        for i in range(N-1):
            eXXpYY(param[step, 1, i],wires = w[i: i+2])
            eYXmXY(param[step,2, i],wires = w[i: i+2])
            eM3(param[step, 3, i], wires = w[i:i+2])
            eYXmXY(param[step,4, i],wires = w[i: i+2])
            eXXpYY(param[step, 5, i],wires = w[i: i+2])
        
        eXXpYY(param[step, 1, N-1],wires = [w[N-1], w[0]])
        eYXmXY(param[step,2, N-1],wires = [w[N-1], w[0]])
        eM3(param[step, 3, N-1], wires = [w[N-1], w[0]])
        eYXmXY(param[step,4, N-1],wires = [w[N-1], w[0]])
        eXXpYY(param[step, 5, N-1],wires = [w[N-1], w[0]])


In [3]:
symbols = ["N", "N"]
dist = 2.0741632451
geometry = np.array([[0.0, 0.0, 0], 
                        [0.0, 0.0, 2.0741632451]], requires_grad = False)
H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, method="pyscf")
N = 20 
p = 1
param = np.random.randn(p, 6, N, requires_grad = True)
print(param.shape)

print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)

(1, 6, 20)
Number of qubits =  20
The Hamiltonian is    (-66.19208134035607) [I0]
+ (1.0090899092682832) [Z19]
+ (1.0090899092682835) [Z18]
+ (1.1672982589937613) [Z16]
+ (1.1672982589937613) [Z17]
+ (1.1672982589937617) [Z14]
+ (1.167298258993762) [Z15]
+ (1.2839292099345494) [Z12]
+ (1.2839292099345494) [Z13]
+ (1.3372787787139824) [Z6]
+ (1.3372787787139824) [Z7]
+ (1.3441573664755404) [Z11]
+ (1.3441573664755406) [Z10]
+ (1.3441573664755413) [Z9]
+ (1.3441573664755417) [Z8]
+ (1.836467968537632) [Z5]
+ (1.8364679685376328) [Z4]
+ (9.607991090184973) [Z2]
+ (9.607991090184973) [Z3]
+ (9.60812198040009) [Z0]
+ (9.60812198040009) [Z1]
+ (0.08338058917218819) [Z6 Z12]
+ (0.08338058917218819) [Z7 Z13]
+ (0.10187523164840515) [Z8 Z14]
+ (0.10187523164840515) [Z9 Z15]
+ (0.1018752316484052) [Z10 Z16]
+ (0.1018752316484052) [Z11 Z17]
+ (0.11343732257525227) [Z6 Z16]
+ (0.11343732257525227) [Z7 Z17]
+ (0.1134373225752523) [Z6 Z14]
+ (0.1134373225752523) [Z7 Z15]
+ (0.11400640234214356) [Z4 

In [4]:
energy_list, param = find_groundstate(N, p, circuit, H, param=param)

(1, 6, 20)
